# Notebook to Assist in the Development of 'extract_fits_headers'

In [1]:
# First import required libraries
import os
import sys
import pandas as pd
import datetime
import math
from astropy.io import fits
#import a wrning suppression library to keep the notebook clean
import warnings
import re
import requests
warnings.filterwarnings('ignore')
# Set option to display all rows
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)  # None means unlimited


### Function Description: `extract_fits_headers`

The `extract_fits_headers` function is designed to extract headers from FITS (Flexible Image Transport System) files located in specified directories. The header files contain information that will be used to create a CSV file having the correct format for uploading to AstroBin

#### How it works:

1. **Input Parameter**:
   - `directories`: A list of strings where each string is a path to a directory. These directories are where the function searches for FITS files.

2. **Process**:
   - The function iterates over each directory in the provided list.
   - Within each directory, it uses `os.walk` to traverse the directory tree.
   - For each file in the directories, the function checks if the file ends with a `.fits` extension, indicating it is a FITS file.
   - When a FITS file is identified, the function attempts to open it using `fits.open` from the `astropy.io.fits` module.
   - The header of the first HDU (Header/Data Unit) of the FITS file is extracted and stored in a dictionary along with the file path.
   - If there are any errors during the reading of a FITS file, the error is caught and printed.
   - This process continues until all FITS files in the given directories have been processed.

3. **Output**:
   - The function returns a Pandas DataFrame.
   - Each row in the DataFrame represents the contents the contents of a FITS header file.
   - The columns of the DataFrame correspond to the headers found in the FITS files, along with an additional column for the file path.


In [2]:
def extract_fits_headers(directories):
    """
    Extract headers from FITS files in given directories and summarize the observation session.

    Parameters:
    directories (list): List of directory paths to search for FITS files.

    Returns:
    DataFrame: Pandas DataFrame containing the headers from the FITS files.
    """
    headers = []

    # Extract headers from FITS files
    for directory in directories:
        for root, _, files in os.walk(directory):
            print('Extracting headers from ', root if root == directory else os.path.basename(root))
            for file in files:
                if file.lower().endswith('.fits'):
                    file_path = os.path.join(root, file)
                    try:
                        with fits.open(file_path, mode='update') as hdul:
                            header = hdul[0].header
                            headers.append(dict(header, file_path=file_path))
                    except Exception as e:
                        print(f"Error reading {file_path}: {e}")

    print('Header extraction complete')
    
    return pd.DataFrame(headers)




## Testing

Supply the paths to a directory or multiple directories that mutliple FITS files. The directory/directories contain the Lights, Dark, Flats and Bias frames associated with an single target.

In [3]:
p1 = '/mnt/HDD_8TB/Preselected/Sadr Region'
p2 = '/mnt/HDD_8TB/Preselected/Calibration data/30th April 2023'
p3 = '/mnt/HDD_8TB/Preselected/Calibration data/20th April 2023/DARK'

#Call the function
headers_df = extract_fits_headers([p1,p2,p3] )

Extracting headers from  /mnt/HDD_8TB/Preselected/Sadr Region
Extracting headers from  OIII
Extracting headers from  11th August
Extracting headers from  10th August
Extracting headers from  8th August
Extracting headers from  9th August
Extracting headers from  Ha
Extracting headers from  6th July
Extracting headers from  7th July
Extracting headers from  5th July
Extracting headers from  SII
Extracting headers from  10th August
Extracting headers from  7th August
Extracting headers from  7th July
Extracting headers from  30th July
Extracting headers from  8th August
Extracting headers from  RGB
Extracting headers from  11th August
Extracting headers from  /mnt/HDD_8TB/Preselected/Calibration data/30th April 2023
Extracting headers from  BIAS
Extracting headers from  FLAT
Extracting headers from  /mnt/HDD_8TB/Preselected/Calibration data/20th April 2023/DARK
Header extraction complete


In [4]:
headers_df.head()

,SIMPLE,BITPIX,NAXIS,NAXIS1,NAXIS2,EXTEND,BZERO,IMAGETYP,EXPOSURE,EXPTIME,...,ROWORDER,EQUINOX,SWCREATE,BORTLE,SQM,IMSCALE,HFR,FWHM,file_path,DEWPOINT
0,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,TOP-DOWN,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.4,1.436216,1.67,2.398480,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-10_Time_23-30-56_Filter_OIII_Exposure_600.00s_HFR_1.67pxs_FrameNo_0005.fits,NaN
1,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,TOP-DOWN,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.4,1.436216,1.68,2.412842,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-10_Time_23-52-12_Filter_OIII_Exposure_600.00s_HFR_1.68pxs_FrameNo_0007.fits,NaN
2,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,TOP-DOWN,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.4,1.436216,1.63,2.341031,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-11_Time_00-11-53_Filter_OIII_Exposure_600.00s_HFR_1.63pxs_FrameNo_0008.fits,NaN
3,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,TOP-DOWN,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.4,1.436216,1.66,2.384118,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-10_Time_23-10-52_Filter_OIII_Exposure_600.00s_HFR_1.66pxs_FrameNo_0003.fits,NaN
4,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,TOP-DOWN,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.4,1.436216,1.60,2.297945,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-11_Time_03-51-27_Filter_OIII_Exposure_600.00s_HFR_1.60pxs_FrameNo_0017.fits,NaN


In [5]:
pd.DataFrame(headers_df.iloc[0])

,0
SIMPLE,True
BITPIX,16
NAXIS,2
NAXIS1,9576
NAXIS2,6388
EXTEND,True
BZERO,32768
IMAGETYP,LIGHT
EXPOSURE,600.0
EXPTIME,600.0


## The two function below create and display a summary of the header data collected

### Function Description: `format_seconds_to_hms`

The `format_seconds_to_hms` function is designed to convert a time duration in seconds to a human-readable format, expressing the duration in hours, minutes, and seconds.

#### How it works:

1. **Input Parameter**:
   - `seconds`: An integer representing the total number of seconds to be converted.

2. **Process**:
   - The function divides the total seconds into hours, minutes, and remaining seconds using the `divmod` function.
   - It then constructs a string to represent this time, formatted as 'H hrs M mins S secs'.
   - If the hours or minutes are zero, they are omitted from the string for brevity.

3. **Output**:
   - The function returns a string formatted to represent the time in hours, minutes, and seconds.

#### Purpose and Use:
This function is a healper function called from the function sumerize_ session


In [6]:
def format_seconds_to_hms(seconds):
    """
    Convert seconds to a formatted string in hours, minutes, and seconds.
    
    Parameters:
    seconds (int): Total number of seconds.

    Returns:
    str: Formatted time string in 'H hrs M mins S secs' format.
    """
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)

    time_parts = []
    if hours > 0:
        time_parts.append(f"{hours} hrs")
    if minutes > 0:
        time_parts.append(f"{minutes} mins")
    time_parts.append(f"{seconds:.0F} secs")

    return ' '.join(time_parts)


### Function Description: `summarize_session`

The `summarize_session` function processes a DataFrame containing FITS file headers to summarize an astronomical observation session.

#### How it works:

1. **Input Parameter**:
   - `df`: A Pandas DataFrame that contains headers of FITS files, including fields like `IMAGETYP`, `FILTER`, and `EXPOSURE`.

2. **Process**:
   - The function first checks if the 'IMAGETYP' column exists in the DataFrame.
   - For each type of image (`LIGHT`, `FLAT`, `BIAS`, `DARK`), it groups the DataFrame by the `FILTER` column (for `LIGHT` and `FLAT` types) and calculates the total exposure time and frame count.
   - It then prints a summary for each image type and filter, including the number of frames and the total exposure time, formatted in a human-readable format using `format_seconds_to_hms`.
   - Specifically for `LIGHT` frames, it calculates and prints the total session exposure time.

3. **Output**:
   - This function does not return a value but prints a detailed summary of the observation session directly to the console.


In [7]:
def sumarize_session(df):    
    # Summarize observation session
    if 'IMAGETYP' in df:
        print("\nObservation session Summary:")

        # Process LIGHT, FLAT, BIAS, and DARK frames
        for imagetyp in ['LIGHT', 'FLAT', 'BIAS', 'DARK']:
            if imagetyp in df['IMAGETYP'].values:
                group = df[df['IMAGETYP'] == imagetyp]
                if imagetyp in ['LIGHT', 'FLAT']:
                    print(f"\n{imagetyp}S:")
                    for filter_type, group_df in group.groupby('FILTER'):
                        frame_count = group_df.shape[0]
                        total_exposure = group_df['EXPOSURE'].sum()
                        formatted_time = format_seconds_to_hms(total_exposure)
                        print(f"  Filter {filter_type}:\t {frame_count} frames, Exposure time: {formatted_time}")
                
                # Total exposure for LIGHT frames
                if imagetyp == 'LIGHT':
                    total_light_exposure = group['EXPOSURE'].sum()
                    formatted_total_light_time = format_seconds_to_hms(total_light_exposure)
                    print(f"\nTotal session exposure for LIGHTs:\t {formatted_total_light_time}")
                elif imagetyp != 'FLAT':
                    # Total count and exposure for FLAT, BIAS, and DARK
                    count = group.shape[0]
                    total_exposure = group['EXPOSURE'].sum()
                    formatted_time = format_seconds_to_hms(total_exposure)
                    print(f"\n{imagetyp}:\t {count} frames, Exposure time: {formatted_time}")
    else:
        print("No 'IMAGETYP' column found in headers.")


In [8]:
sumarize_session(headers_df)


Observation session Summary:

LIGHTS:
  Filter Blue:	 17 frames, Exposure time: 17.0 mins 0 secs
  Filter Green:	 18 frames, Exposure time: 18.0 mins 0 secs
  Filter Ha:	 51 frames, Exposure time: 8.0 hrs 30.0 mins 0 secs
  Filter OIII:	 54 frames, Exposure time: 9.0 hrs 0 secs
  Filter Red:	 30 frames, Exposure time: 30.0 mins 0 secs
  Filter SII:	 51 frames, Exposure time: 8.0 hrs 30.0 mins 0 secs

Total session exposure for LIGHTs:	 27.0 hrs 5.0 mins 0 secs

FLATS:
  Filter Blue:	 50 frames, Exposure time: 9 secs
  Filter Green:	 50 frames, Exposure time: 8 secs
  Filter Ha:	 50 frames, Exposure time: 23 secs
  Filter Lum:	 50 frames, Exposure time: 2 secs
  Filter OIII:	 50 frames, Exposure time: 25 secs
  Filter Red:	 50 frames, Exposure time: 6 secs
  Filter SII:	 50 frames, Exposure time: 28 secs

BIAS:	 200 frames, Exposure time: 0 secs

DARK:	 100 frames, Exposure time: 9.0 hrs 10.0 mins 0 secs


### Function Description: `create_calibration_df`

Now we have all the headers in a single data frame, code is developed to analyse this. The first thing is to count the calabration frames.

The `create_calibration_df` function processes a DataFrame containing FITS file headers and generates a summarized DataFrame specifically for calibration data. It focuses on specific types of images used in astronomical observations, namely DARK, BIAS, FLAT, and FLATDARKS frames.

#### How it works:

1. **Input Parameter**:
   - `df`: A Pandas DataFrame that includes headers from FITS files. Key columns in this DataFrame are 'IMAGETYP', 'GAIN', and optionally 'FILTER'.

2. **Process**:
   - The function first filters the DataFrame to include only rows where the 'IMAGETYP' matches the relevant types (DARK, BIAS, FLAT, and FLATDARKS).
   - It then checks if the 'FILTER' column exists. If it does, the function sets the 'FILTER' value to an empty string for all types except FLAT. For FLAT frames, the actual filter value is retained.
   - The data is then grouped by 'IMAGETYP', 'GAIN', and 'FILTER' (for FLAT frames). For other frame types, 'FILTER' is effectively ignored due to the empty string assignment.
   - The function counts the number of occurrences for each group, effectively summarizing the number of each type of frame for each gain setting (and filter type for FLAT frames).

3. **Output**:
   - The function returns a new Pandas DataFrame with columns 'TYPE', 'GAIN', 'FILTER' (where applicable), and 'NUMBER'.
   - 'TYPE' corresponds to 'IMAGETYP', 'NUMBER' is the count of frames for each group, and 'GAIN' and 'FILTER' are taken from the original DataFrame.


In [9]:
def create_calibration_df(df):
    """
    Creates a DataFrame for calibration data based on specific IMAGETYPs, GAIN, and FILTER.
    
    Parameters:
    df (pandas.DataFrame): The DataFrame containing FITS header data.
    
    Returns:
    pandas.DataFrame: A DataFrame with columns 'TYPE', 'GAIN', 'FILTER' (if applicable), and 'NUMBER'.
    """
    relevant_types = ['DARK', 'BIAS', 'FLAT', 'FLATDARKS']
    filtered_df = df[df['IMAGETYP'].isin(relevant_types)]

    # Group by IMAGETYP and GAIN, and additionally by FILTER for FLAT
    if 'FILTER' in df.columns:
        filtered_df.loc[filtered_df['IMAGETYP'] != 'FLAT', 'FILTER'] = ''
        group_counts = filtered_df.groupby(['IMAGETYP', 'GAIN', 'FILTER']).size().reset_index(name='NUMBER')
    else:
        group_counts = filtered_df.groupby(['IMAGETYP', 'GAIN']).size().reset_index(name='NUMBER')

    return group_counts.rename(columns={'IMAGETYP': 'TYPE'})


In [10]:
calibration_df = create_calibration_df(headers_df)

In [11]:
calibration_df

,TYPE,GAIN,FILTER,NUMBER
0,BIAS,0,,100
1,BIAS,100,,100
2,DARK,0,,50
3,DARK,100,,50
4,FLAT,0,Blue,50
5,FLAT,0,Green,50
6,FLAT,0,Lum,50
7,FLAT,0,Red,50
8,FLAT,100,Ha,50
9,FLAT,100,OIII,50


ext we want to develop a function that extracts all the LIGHT frames from the DataFrame containing all the FITS header files.

The `create_Lights_df` function processes a DataFrame containing FITS file headers and generates a summarized DataFrame specifically for LIGHTS data. 

#### How it works:

1. **Input Parameters**:
   - `df`: A Pandas DataFrame that includes headers from FITS files.
   - 'calibration_df': A Pandas DataFrame that provides the number of all types of calibration frames found by the function `create_calibration_df`

2. **Process**:
   - The function first filters the DataFrame to include only rows where the 'IMAGETYP' matches the LIGHT type and creates a copy.
   - It then creates a DATE column which is a datatime formatted version of the DATE-LOC column

3. **Output**:
   - The function returns a new Pandas DataFrame that holds all the LIGHT frame header information 


In [12]:
def create_lights_df(df: pd.DataFrame, calibration_df: pd.DataFrame)-> pd.DataFrame:
                 
    """
    Creates a DataFrame for 'LIGHT' type data 
    Args:
        df (pd.DataFrame): DataFrame containing FITS header data.
        calibration_df (pd.DataFrame): DataFrame containing calibration data.


    Returns:
        pd.DataFrame: Aggregated DataFrame with 'LIGHT' type data.
    """
    light_df = df[df['IMAGETYP'] == 'LIGHT'].copy()
    light_df['DATE'] = pd.to_datetime(light_df['DATE-LOC']).dt.date

    return pd.DataFrame(light_df)


Create lights dataframe and show example of a single header file from the data frame

In [13]:
lights_df =create_lights_df(headers_df,calibration_df)
pd.DataFrame(lights_df.iloc[0])

,0
SIMPLE,True
BITPIX,16
NAXIS,2
NAXIS1,9576
NAXIS2,6388
EXTEND,True
BZERO,32768
IMAGETYP,LIGHT
EXPOSURE,600.0
EXPTIME,600.0


### Function: `get_bortle_sqm`

Most of the parameters required for the AstroBin upload can be obtained or derived from the header file information. The Bortle and SQM parameters, however, have to be supplied from and external source. Here we develp code that retrieves the artificial_brightness of the sky at a given latitude and longtitude from the web resource https://www.lightpollutionmap.info. The API_KEY and API_ENDPOINT for this service are stored in the secret.csv file. The `get_bortle_sqm` function is designed to `get_bortle_sqm` function is designed to retrieve the Bortle scale classification and the SQM (Sky Quality Meter) value for a specified geographic location. It then converts the result into SQM in $\text{mag} \cdot \text{arcsec}^{-2}$ as well as the Bortle scale. The Bortle scale is a nine-level numeric scale that quantifies the observability of celestial objects (like stars and planets) and the impact of light pollution on the night sky. Bortle scale is provided by the helper function `sqm_to_bortle`

#### Parameters:

- `lat` (float): Latitude of the location.
- `lon` (float): Longitude of the location.
- `api_key` (str): An API key required to access light pollution data from a specific API.
- `api_endpoint` (str): The URL of the API endpoint from which light pollution data is retrieved.

#### Returns:

- A tuple containing:
  - The Bortle scale classification (integer, range: 1 to 9, where 1 is the darkest and 9 is the brightest).
  - The SQM value (float, rounded to two decimal places), representing the sky brightness in magnitudes per square arcsecond ($\text{mag} \cdot \text{arcsec}^{-2}$ )

In [14]:
def get_bortle_sqm(lat, lon, api_key, api_endpoint):
    """
    Retrieves the Bortle scale classification and SQM (Sky Quality Meter) value for a given latitude and longitude.

    This function makes a request to a specified API endpoint using the provided API key. It fetches the artificial
    brightness for the specified coordinates and calculates the SQM. The Bortle scale classification is then determined
    based on the SQM value.

    Args:
    lat (float): Latitude of the location.
    lon (float): Longitude of the location.
    api_key (str): API key for accessing the light pollution data.
    api_endpoint (str): URL of the API endpoint to retrieve light pollution data.

    Returns:
    tuple: A tuple containing the Bortle scale classification (int) and the SQM value (float), rounded to two decimal places.
    """
    params = {
        'ql': 'wa_2015',
        'qt': 'point',
        'qd': f'{lon},{lat}',
        'key': api_key
    }
    response = requests.get(api_endpoint, params=params)
    response.raise_for_status()
    artificial_brightness = float(response.text)
    sqm = (math.log10((artificial_brightness + 0.171168465)/108000000)/-0.4)
    bortle_class = sqm_to_bortle(sqm)
    return bortle_class, round(sqm, 2)

def sqm_to_bortle(sqm):
    """
    Converts an SQM (Sky Quality Meter) value to the corresponding Bortle scale classification.

    The Bortle scale is used to quantify the astronomical observability of celestial objects, affected by light pollution.

    Args:
    sqm (float): The SQM value indicating the level of light pollution.

    Returns:
    int: The Bortle scale classification (ranging from 1 to 9), where 1 indicates the darkest skies and 9 the brightest.
    """
    if sqm > 21.99:
        return 1
    elif 21.50 <= sqm <= 21.99:
        return 2
    elif 21.25 <= sqm <= 21.49:
        return 3
    elif 20.50 <= sqm <= 21.24:
        return 4
    elif 19.50 <= sqm <= 20.49:
        return 5
    elif 18.50 <= sqm <= 19.49:
        return 6
    elif 17.50 <= sqm <= 18.49:
        return 7
    elif 17.00 <= sqm <= 17.49:
        return 8
    else:
        return 9


Testing get_bortle_sqm

In [15]:
#Read secrets.cv to retrieve API_KEY and API_ENDPOINT
secret = pd.read_csv('secret.csv')
api_key = secret['API Key'].iloc[0]
api_endpoint = secret['API Endpoint'].iloc[0]
#define latitude and longtitude
lat  = 52.248472
lon  = -0.123167
#call function
bortle, sqm = get_bortle_sqm(lat, lon, api_key,api_endpoint)
print(f"\nRetrieved Bortle of {bortle} and SQM of {sqm} mag/arc-seconds^2 for lat {lat}, lon {lon} from API at {api_endpoint}.\n")


Retrieved Bortle of 4 and SQM of 20.52 mag/arc-seconds^2 for lat 52.248472, lon -0.123167 from API at https://www.lightpollutionmap.info/QueryRaster/.



### Function: `calculate_auxiliary_parameters`

This function is designed calculate parameters that are not contained in the FITS header file information.

#### Functionality:

- **Bortle Scale and SQM Calculation**: The function calculates the Bortle scale classification and SQM (Sky Quality Meter) values for each LIGHTS file. Th ecode reads teh latitude and longtitude values for the image from the FITS header. This is compared with entries in the sites.csv file. If there is a match the file parameters are used. If not the data is fetched from an API.
- **HFR, IMSCALE, and FWHM Computation**: It also computes HFR (Half Flux Radius), IMSCALE (Image Scale), and FWHM (Full Width Half Maximum) for each observation in the DataFrame.

#### Parameters:

- `df` (pd.DataFrame): The DataFrame to be processed. It should contain the following columns:
  - `SITELAT`: Latitude of the observation site.
  - `SITELONG`: Longitude of the observation site.
  - `file_path`: Path to the FITS file containing the observation data.
  - `XPIXSZ`: Pixel size of the camera used (in micrometers).
  - `FOCALLEN`: Focal length of the telescope (in millimeters).

#### Returns:

- lights_df a pd.DataFrame with additional columns for the calculated parameters.

#### Additional Notes:

- The function relies on 'secret.csv' for the API key and endpoint. If this file is not found, manual data entry for Bortle and SQM values is required.
- The 'sites.csv' file is used to store and retrieve existing Bortle and SQM values for known latitude and longitude pairs. If a new pair of coordinates is   detected then the SQM and Bortle values for this site are calculated and stored in the sites.csv file.
- If the a valid API_KEY is not found the web resource cannot be used in this case the relevant parameters may be entered in the sites.csv file manually. 
- The calculations of HFR, IMSCALE, and FWHM are based on regex matching and basic astronomical formulas.

In [16]:
def calculate_auxiliary_parameters(df):
    """
    Calculates and updates auxiliary parameters for each row in a DataFrame based on astronomical observation data.

    This function processes a DataFrame containing astronomical observation data. It updates each row with Bortle scale,
    SQM (Sky Quality Meter) values, HFR (Half Flux Radius), IMSCALE (Image Scale), and FWHM (Full Width Half Maximum). 
    The Bortle scale and SQM values are either fetched from an API (using latitude and longitude from the DataFrame) 
    or retrieved from a local 'sites.csv' file if already available. The function also calculates HFR, IMSCALE, and FWHM
    based on provided FITS file information in the DataFrame.

    Args:
    df (pd.DataFrame): A DataFrame containing FITS file data. Expected to have columns 'SITELAT', 'SITELONG', 'file_path',
                       'XPIXSZ', and 'FOCALLEN'.

    Returns:
    pd.DataFrame: The input DataFrame with additional columns for Bortle scale, SQM, HFR, IMSCALE, and FWHM.
                  These columns are named 'BORTLE', 'SQM', 'HFR', 'IMSCALE', and 'FWHM', respectively.

    Notes:
    - The function reads API key and endpoint from 'secret.csv'.
    - If 'secret.csv' is not found, it requires manual data entry for Bortle and SQM in 'configuration.csv'.
    - The 'sites.csv' file is used to store and retrieve existing Bortle and SQM values for known locations.
    - HFR, IMSCALE, and FWHM calculations are based on regex matching and basic astronomical formulas.
    """


    try:
        sites = pd.read_csv('sites.csv')
    except FileNotFoundError:
        sites = pd.DataFrame(columns=['Latitude', 'Longitude', 'Bortle', 'SQM'])

    try:
        secret = pd.read_csv('secret.csv')
        api_key = secret['API Key'].iloc[0]
        api_endpoint = secret['API Endpoint'].iloc[0]
    except FileNotFoundError:
        print("Secret file 'secret.csv' not found. Please provide API key and endpoint.")
        api_key = None
        api_endpoint = None

    processed_sites = set()  # Set to keep track of processed lat-lon pairs

    for index, row in df.iterrows():
        lat, lon = round(row['SITELAT'],3), round(row['SITELONG'],3)
        
        site_data = sites[(sites['Latitude'] == lat) & (sites['Longitude'] == lon)]
        if site_data.empty and api_key:
            try:
                bortle, sqm = get_bortle_sqm(lat, lon, api_key,api_endpoint)
                new_site = {'Latitude': lat, 'Longitude': lon, 'Bortle': bortle, 'SQM': sqm}
                new_site_df = pd.DataFrame([new_site])
                sites = pd.concat([sites, new_site_df], ignore_index=True)
                sites.to_csv('sites.csv', index=False)
                df.at[index, 'BORTLE'] = bortle
                df.at[index, 'SQM'] = sqm
                print(f"Retrieved Bortle of {bortle} and SQM of {sqm} mag/arc-seconds^2 for lat {lat}, lon {lon} from API at {api_endpoint}.")
            except requests.exceptions.RequestException as e:
                print(f"Failed to retrieve data from the API for lat {lat}, lon {lon}: {e}")
                df.at[index, 'BORTLE'] = 0
                df.at[index, 'SQM'] = 0
        elif not site_data.empty:
            if (lat, lon) not in processed_sites:
                processed_sites.add((lat, lon))
                bortle, sqm = site_data.iloc[0]['Bortle'], site_data.iloc[0]['SQM']
                print(f"\nUsed existing Bortle of {bortle} and SQM of {sqm} mag/arc-seconds^2 for lat {lat}, lon {lon} from sites.csv.")
               
            bortle, sqm = site_data.iloc[0]['Bortle'], site_data.iloc[0]['SQM']
            df.at[index, 'BORTLE'] = bortle
            df.at[index, 'SQM'] = sqm
        else:
            if (lat, lon) not in processed_sites:
                processed_sites.add((lat, lon))
                print(f"\nNo data found for lat {lat}, lon {lon}. Set Bortle and SQM to 0.")
            df.at[index, 'BORTLE'] = 0
            df.at[index, 'SQM'] = 0


        file_path = row['file_path']
        hfr_match = re.search(r'HFR_([0-9.]+)', file_path)
        hfr = float(hfr_match.group(1)) if hfr_match else None
        imscale = row['XPIXSZ'] / row['FOCALLEN'] * 206.265
        fwhm = hfr * imscale if hfr else None

        df.at[index, 'HFR'] = hfr
        df.at[index, 'IMSCALE'] = imscale
        df.at[index, 'FWHM'] = fwhm

    print('\nCompleted sky quality extraction')
    return df.round(2)


In [17]:
lights_df = calculate_auxiliary_parameters(lights_df)


Used existing Bortle of 4.0 and SQM of 20.52 mag/arc-seconds^2 for lat 52.248, lon -0.123 from sites.csv.

Completed sky quality extraction


In [18]:
lights_df.head(10)

,SIMPLE,BITPIX,NAXIS,NAXIS1,NAXIS2,EXTEND,BZERO,IMAGETYP,EXPOSURE,EXPTIME,...,EQUINOX,SWCREATE,BORTLE,SQM,IMSCALE,HFR,FWHM,file_path,DEWPOINT,DATE
0,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.52,1.44,1.67,2.40,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-10_Time_23-30-56_Filter_OIII_Exposure_600.00s_HFR_1.67pxs_FrameNo_0005.fits,NaN,2023-08-10
1,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.52,1.44,1.68,2.41,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-10_Time_23-52-12_Filter_OIII_Exposure_600.00s_HFR_1.68pxs_FrameNo_0007.fits,NaN,2023-08-10
2,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.52,1.44,1.63,2.34,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-11_Time_00-11-53_Filter_OIII_Exposure_600.00s_HFR_1.63pxs_FrameNo_0008.fits,NaN,2023-08-11
3,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.52,1.44,1.66,2.38,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-10_Time_23-10-52_Filter_OIII_Exposure_600.00s_HFR_1.66pxs_FrameNo_0003.fits,NaN,2023-08-10
4,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.52,1.44,1.60,2.30,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-11_Time_03-51-27_Filter_OIII_Exposure_600.00s_HFR_1.60pxs_FrameNo_0017.fits,NaN,2023-08-11
5,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.52,1.44,1.67,2.40,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-10_Time_23-42-10_Filter_OIII_Exposure_600.00s_HFR_1.67pxs_FrameNo_0006.fits,NaN,2023-08-10
6,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.52,1.44,1.64,2.36,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-10_Time_22-49-42_Filter_OIII_Exposure_600.00s_HFR_1.64pxs_FrameNo_0001.fits,NaN,2023-08-10
7,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.52,1.44,1.61,2.31,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-11_Time_02-40-31_Filter_OIII_Exposure_600.00s_HFR_1.61pxs_FrameNo_0010.fits,NaN,2023-08-11
8,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.52,1.44,1.66,2.38,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-10_Time_23-20-55_Filter_OIII_Exposure_600.00s_HFR_1.66pxs_FrameNo_0004.fits,NaN,2023-08-10
9,True,16,2,9576,6388,True,32768,LIGHT,600.0,600.0,...,2000.0,N.I.N.A. 2.3.0.2001,4.0,20.52,1.44,1.65,2.37,/mnt/HDD_8TB/Preselected/Sadr Region/OIII/11th August/Sadr Region_Date_2023-08-10_Time_22-59-43_Filter_OIII_Exposure_600.00s_HFR_1.65pxs_FrameNo_0002.fits,NaN,2023-08-10


Show example of a single header file from the lights_df data frame

In [19]:
pd.DataFrame(lights_df.iloc[0])

,0
SIMPLE,True
BITPIX,16
NAXIS,2
NAXIS1,9576
NAXIS2,6388
EXTEND,True
BZERO,32768
IMAGETYP,LIGHT
EXPOSURE,600.0
EXPTIME,600.0


### Function `aggregate_parameters`

#### Purpose:

`aggregate_parameters` is a function designed to aggregate all relevant data contained in both the lights_df and calibration_df into a single data frame that is required by AstroBin for upload to its site.

#### Functionality:

1. **Data Aggregation**:
   - Groups `lights_df` data by `date`, `filter`, `gain`, `xbinning`, and `exposure`.
   - Calculates mean values for sensor cooling, temperature, and mean FWHM.
   - Aggregates observational data to provide a concise overview.

2. **Calibration Data Integration**:
   - Includes relevant calibration data such as darks, flats, bias, and flatDarks from `calibration_df`.
   - Matches calibration data based on filter and gain settings.
   - Ensures that each observation row is supplemented with the appropriate calibration data.

3. **Additional Parameters**:
   - Incorporates Bortle scale and mean SQM values into the aggregated data.
   - Rounds off sensor cooling data for clarity.

4. **Return Value**:
   - Returns a Pandas DataFrame with all aggregated and computed parameters, rounded to two decimal places for readability.

In [20]:
def aggregate_parameters(lights_df, calibration_df):
    """
    Aggregates observational parameters from lights_df and calibration data from calibration_df.

    The function aggregates data by date, filter, gain, xbinning, and exposure. It computes the mean values
    for sensor cooling, temperature, and mean FWHM. It also integrates calibration data (darks, flats, bias,
    flatDarks) based on the type of filter and gain settings. Additionally, it includes Bortle scale and SQM
    values from the lights_df into the aggregated data.

    Parameters:
    lights_df (pd.DataFrame): DataFrame containing observational data with FITS headers.
    calibration_df (pd.DataFrame): DataFrame containing calibration data like darks, flats, etc.

    Returns:
    pd.DataFrame: Aggregated DataFrame with observational and calibration parameters, 
                  including sensor cooling, temperature, mean FWHM, and more.

    Notes:
    - The function expects 'lights_df' to contain columns like 'date', 'filter', 'gain', etc.
    - The 'calibration_df' is expected to contain calibration data corresponding to 'lights_df'.
    """

    
    lights_df.columns = lights_df.columns.str.lower()

    aggregated_df = lights_df.groupby(['date', 'filter', 'gain', 'xbinning', 'exposure']).agg(
            number=('date', 'count'),
            sensorCooling=('ccd-temp', 'mean'),
            temperature=('foctemp', 'mean'),
            meanFwhm = ('fwhm', 'mean')
        ).reset_index()
    aggregated_df.rename(columns={'xbinning': 'binning',
                                  'exposure': 'duration'}, inplace=True)

    
    
    def get_calibration_data(row: pd.Series, cal_type: str, calibration_df: pd.DataFrame) -> int:
        # For FLAT type, match both 'GAIN' and 'FILTER'
        if cal_type == 'FLAT':
            match = calibration_df[(calibration_df['TYPE'] == cal_type) & 
                                   (calibration_df['GAIN'] == row['gain']) &
                                   (calibration_df['FILTER'].str.upper() == row['filter'].upper())]
        else:
            match = calibration_df[(calibration_df['TYPE'] == cal_type) & 
                                   (calibration_df['GAIN'] == row['gain'])]
    
        return match['NUMBER'].sum() if not match.empty else 0
    
    # Then apply it to your aggregated DataFrame
    aggregated_df['darks'] = aggregated_df.apply(get_calibration_data, args=('DARK', calibration_df), axis=1)
    aggregated_df['flats'] = aggregated_df.apply(get_calibration_data, args=('FLAT', calibration_df), axis=1)
    aggregated_df['bias'] = aggregated_df.apply(get_calibration_data, args=('BIAS', calibration_df), axis=1)
    aggregated_df['flatDarks'] = aggregated_df.apply(get_calibration_data, args=('FLATDARKS', calibration_df), axis=1)

    aggregated_df['bortle'] = lights_df['bortle']
    aggregated_df['meanSqm'] = lights_df['sqm']

    aggregated_df['sensorCooling'] = aggregated_df['sensorCooling'].round().astype(int)

    return aggregated_df.round(2)

In [21]:
aggregated_df =aggregate_parameters(lights_df,calibration_df)

In [22]:
aggregated_df

,date,filter,gain,binning,duration,number,sensorCooling,temperature,meanFwhm,darks,flats,bias,flatDarks,bortle,meanSqm
0,2023-07-06,Ha,100,1,600.0,15,-10,10.24,2.37,50,50,100,0,4.0,20.52
1,2023-07-07,Ha,100,1,600.0,24,-10,11.51,2.38,50,50,100,0,4.0,20.52
2,2023-07-08,Ha,100,1,600.0,12,-10,17.47,2.39,50,50,100,0,4.0,20.52
3,2023-07-08,SII,100,1,600.0,5,-10,16.82,2.46,50,50,100,0,4.0,20.52
4,2023-07-29,SII,100,1,600.0,2,-10,13.40,2.40,50,50,100,0,4.0,20.52
5,2023-07-30,SII,100,1,600.0,1,-10,12.80,2.40,50,50,100,0,4.0,20.52
6,2023-08-06,SII,100,1,600.0,8,-10,11.41,2.41,50,50,100,0,4.0,20.52
7,2023-08-07,SII,100,1,600.0,22,-10,9.53,2.43,50,50,100,0,4.0,20.52
8,2023-08-08,OIII,100,1,600.0,7,-10,9.56,2.38,50,50,100,0,4.0,20.52
9,2023-08-08,SII,100,1,600.0,5,-10,9.86,2.45,50,50,100,0,4.0,20.52


Astrobin requires the user provide a 4 digit code that represents the filter being used. The code can be obtained from the url in the equipment section for the filter being used. For instance for an Astronomik H-alpha CCD 6nm 2 inch filter the [equipment explorer url](https://app.astrobin.com/equipment/explorer/filter?page=1) is:

https://app.astrobin.com/equipment/explorer/filter/4663/astronomik-h-alpha-ccd-6nm-2

The code in this case is 4663.

The filter mappings are saved in the filters.csv file which are read in to the filters_df dataframe. For reference the mappings for my Astronomik filter set are shown below:

|Filter|Code|
|------|----|
|Ha    |4663|
|SII   |4844|
|OIII  |4752|
|Red   |4649|
|Green |4643|
|Blue  |4637|
|Lum   |2906|
|CLS   |4061|


Read the filter mappings

In [23]:
try:
    filter_df = pd.read_csv('filters.csv')
except FileNotFoundError:
    print('filter.csv not found')


### Function `create_astrobin_output`

#### Purpose:

`create_astrobin_output` is tailored for transforming aggregated astronomical observational data into a format that is compatible with AstroBin. The function facilitates the transition from raw observational data to a structured, standardized format required by AstroBin.

#### Functionality:

1. **Filter Mapping**:
   - Utilizes `filter_df` to map filter names to corresponding codes. 
   - Inserts the mapped filter codes into `aggregated_df` right after the original 'filter' column.

2. **Data Transformation**:
   - Reorders columns to align with the format expected by AstroBin.
   - Drops the original 'filter' column and renames the 'code' column back to 'filter'.
   - Rounds off all numerical data to two decimal places for clarity and consistency.

3. **Output Generation**:
   - Generates a new DataFrame that is structured and ready to be uploaded or utilized on AstroBin.

4. **Return Value**:
   - Returns a Pandas DataFrame formatted specifically for AstroBin, ensuring that the data is clean, concise, and in the correct order.

In [24]:
def create_astrobin_output(aggregated_df, filter_df):
    """
    Transforms aggregated observational data into a format suitable for AstroBin output.

    This function maps filter names to their corresponding codes using 'filter_df', 
    reorders columns to match the expected AstroBin format, and renames the 'code' 
    column back to 'filter'. The function also drops the original 'filter' column 
    and ensures the final DataFrame is rounded to two decimal places for neatness.

    Parameters:
    aggregated_df (pd.DataFrame): DataFrame containing aggregated observational data.
    filter_df (pd.DataFrame): DataFrame that maps filter names to their corresponding codes.

    Returns:
    pd.DataFrame: Transformed DataFrame in the format suitable for AstroBin, with columns reordered 
                  and filter names replaced by their codes.

    Notes:
    - The function assumes that 'aggregated_df' contains all necessary columns for AstroBin output.
    - The 'filter_df' is used to map filter names to codes.
    """
    #Create a mapping Series
    filter_to_code = filter_df.set_index('filter')['code']
    # Find the position of the 'filter' column
    filter_col_index = aggregated_df.columns.get_loc('filter')
    # Insert the 'code' column right after 'filter'
    aggregated_df.insert(filter_col_index + 1, 'code', aggregated_df['filter'].map(filter_to_code))
    column_order = ['date', 'code', 'number', 'duration', 'binning', 'gain', 
                    'sensorCooling', 'darks', 'flats', 'flatDarks', 'bias', 'bortle',
                    'meanSqm', 'meanFwhm', 'temperature']
    df = aggregated_df.copy().drop('filter',axis=1).reindex(columns=column_order).round(2)
    df.rename(columns={'code': 'filter'}, inplace=True)
    return df 

In [25]:
astroBin_df =create_astrobin_output(aggregated_df, filter_df) 

In [26]:
astroBin_df

,date,filter,number,duration,binning,gain,sensorCooling,darks,flats,flatDarks,bias,bortle,meanSqm,meanFwhm,temperature
0,2023-07-06,4663,15,600.0,1,100,-10,50,50,0,100,4.0,20.52,2.37,10.24
1,2023-07-07,4663,24,600.0,1,100,-10,50,50,0,100,4.0,20.52,2.38,11.51
2,2023-07-08,4663,12,600.0,1,100,-10,50,50,0,100,4.0,20.52,2.39,17.47
3,2023-07-08,4844,5,600.0,1,100,-10,50,50,0,100,4.0,20.52,2.46,16.82
4,2023-07-29,4844,2,600.0,1,100,-10,50,50,0,100,4.0,20.52,2.40,13.40
5,2023-07-30,4844,1,600.0,1,100,-10,50,50,0,100,4.0,20.52,2.40,12.80
6,2023-08-06,4844,8,600.0,1,100,-10,50,50,0,100,4.0,20.52,2.41,11.41
7,2023-08-07,4844,22,600.0,1,100,-10,50,50,0,100,4.0,20.52,2.43,9.53
8,2023-08-08,4752,7,600.0,1,100,-10,50,50,0,100,4.0,20.52,2.38,9.56
9,2023-08-08,4844,5,600.0,1,100,-10,50,50,0,100,4.0,20.52,2.45,9.86


Now output the result to CSV file. The contents of this file can me copied and pasted into the AstroBin upload CSV dialog

In [27]:
output_csv = "acquisition.csv"
astroBin_df.to_csv(output_csv , index=False)
print(f"Data exported to {output_csv}")

Data exported to acquisition.csv
